In [1]:
import cv2
import numpy as np
import pandas as pd

from joblib import load

name = input('input name: ')
index = pd.read_csv('../datasets/' + name + '-index.csv', sep=',')
label_text = index.values[:,1]

cap = cv2.VideoCapture(0)
detector = cv2.CascadeClassifier('../models/haarcascades/haarcascade_frontalface_default.xml')
pipeline = load('../models/svm-pca-' + name + '.mdl')

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        img = frame.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = detector.detectMultiScale(gray, 1.2, 4)

        for (x, y, w, h) in faces:
            images = []
            face = gray[y:y+h, x:x+w]
            rsz_face = cv2.resize(face, (64,64), interpolation=cv2.INTER_LINEAR)            
            images.append(np.ravel(rsz_face))

            y_p = pipeline.predict(images)
            prob = pipeline.predict_proba(images)
            max_prob = max(prob[0])
            text = label_text[int(y_p)] + ' ({0:.2g}'.format(max_prob * 100) + '%)'

            cv2.putText(img, text, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), thickness=2)
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.imshow('Face Recognition System', img)

        if cv2.waitKey(1) & 0xFF == ord(' '):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()